<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Ingrid", "transactions": [{"transaction-id": 27, "amount": 3625}, {"transaction-id": 184, "amount": 3349}, {"transaction-id": 435, "amount": 3467}, {"transaction-id": 782, "amount": 3332}, {"transaction-id": 888, "amount": 3134}, {"transaction-id": 1156, "amount": 3635}, {"transaction-id": 1299, "amount": 3227}, {"transaction-id": 1763, "amount": 3259}, {"transaction-id": 2148, "amount": 3426}, {"transaction-id": 2211, "amount": 2958}, {"transaction-id": 2275, "amount": 3494}, {"transaction-id": 2598, "amount": 3236}, {"transaction-id": 2695, "amount": 3242}, {"transaction-id": 2732, "amount": 3360}, {"transaction-id": 3167, "amount": 3555}, {"transaction-id": 3201, "amount": 3128}, {"transaction-id": 3270, "amount": 3365}, {"transaction-id": 3353, "amount": 3493}, {"transaction-id": 3431, "amount": 2996}, {"transaction-id": 3488, "amount": 3364}, {"transaction-id": 4015, "amount": 3277}, {"transaction-id": 4236, "amount": 3191}, {"transaction-id": 4361, "amount": 

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Ingrid", "transactions": [{"transaction-id": 27, "amount": 3625}, {"transaction-id": 184, "amount": 3349}, {"transaction-id": 435, "amount": 3467}, {"transaction-id": 782, "amount": 3332}, {"transaction-id": 888, "amount": 3134}, {"transaction-id": 1156, "amount": 3635}, {"transaction-id": 1299, "amount": 3227}, {"transaction-id": 1763, "amount": 3259}, {"transaction-id": 2148, "amount": 3426}, {"transaction-id": 2211, "amount": 2958}, {"transaction-id": 2275, "amount": 3494}, {"transaction-id": 2598, "amount": 3236}, {"transaction-id": 2695, "amount": 3242}, {"transaction-id": 2732, "amount": 3360}, {"transaction-id": 3167, "amount": 3555}, {"transaction-id": 3201, "amount": 3128}, {"transaction-id": 3270, "amount": 3365}, {"transaction-id": 3353, "amount": 3493}, {"transaction-id": 3431, "amount": 2996}, {"transaction-id": 3488, "amount": 3364}, {"transaction-id": 4015, "amount": 3277}, {"transaction-id": 4236, "amount": 3191}, {"transaction-id": 4361, "amount": 

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Ingrid',
  'transactions': [{'transaction-id': 27, 'amount': 3625},
   {'transaction-id': 184, 'amount': 3349},
   {'transaction-id': 435, 'amount': 3467},
   {'transaction-id': 782, 'amount': 3332},
   {'transaction-id': 888, 'amount': 3134},
   {'transaction-id': 1156, 'amount': 3635},
   {'transaction-id': 1299, 'amount': 3227},
   {'transaction-id': 1763, 'amount': 3259},
   {'transaction-id': 2148, 'amount': 3426},
   {'transaction-id': 2211, 'amount': 2958},
   {'transaction-id': 2275, 'amount': 3494},
   {'transaction-id': 2598, 'amount': 3236},
   {'transaction-id': 2695, 'amount': 3242},
   {'transaction-id': 2732, 'amount': 3360},
   {'transaction-id': 3167, 'amount': 3555},
   {'transaction-id': 3201, 'amount': 3128},
   {'transaction-id': 3270, 'amount': 3365},
   {'transaction-id': 3353, 'amount': 3493},
   {'transaction-id': 3431, 'amount': 2996},
   {'transaction-id': 3488, 'amount': 3364},
   {'transaction-id': 4015, 'amount': 3277},
   {'transacti

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 45,
  'name': 'Alice',
  'transactions': [{'transaction-id': 44, 'amount': 2221},
   {'transaction-id': 116, 'amount': 1922},
   {'transaction-id': 211, 'amount': 1793},
   {'transaction-id': 265, 'amount': 2403},
   {'transaction-id': 354, 'amount': 1762},
   {'transaction-id': 651, 'amount': 2663},
   {'transaction-id': 660, 'amount': 1744},
   {'transaction-id': 800, 'amount': 2315},
   {'transaction-id': 929, 'amount': 2079},
   {'transaction-id': 985, 'amount': 1979},
   {'transaction-id': 1048, 'amount': 2206},
   {'transaction-id': 1276, 'amount': 2487},
   {'transaction-id': 1313, 'amount': 2431},
   {'transaction-id': 1458, 'amount': 1843},
   {'transaction-id': 1478, 'amount': 2076},
   {'transaction-id': 1561, 'amount': 1644},
   {'transaction-id': 1587, 'amount': 2422},
   {'transaction-id': 2198, 'amount': 2144},
   {'transaction-id': 2415, 'amount': 2082},
   {'transaction-id': 2765, 'amount': 1208},
   {'transaction-id': 2894, 'amount': 1735},
   {'transaction-id

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 74},
 {'name': 'Alice', 'count': 5},
 {'name': 'Alice', 'count': 22},
 {'name': 'Alice', 'count': 20},
 {'name': 'Alice', 'count': 14})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(74, 5, 22, 20, 14)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

49.806666666666665

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 44, 'amount': 2221},
  {'transaction-id': 116, 'amount': 1922},
  {'transaction-id': 211, 'amount': 1793},
  {'transaction-id': 265, 'amount': 2403},
  {'transaction-id': 354, 'amount': 1762},
  {'transaction-id': 651, 'amount': 2663},
  {'transaction-id': 660, 'amount': 1744},
  {'transaction-id': 800, 'amount': 2315},
  {'transaction-id': 929, 'amount': 2079},
  {'transaction-id': 985, 'amount': 1979},
  {'transaction-id': 1048, 'amount': 2206},
  {'transaction-id': 1276, 'amount': 2487},
  {'transaction-id': 1313, 'amount': 2431},
  {'transaction-id': 1458, 'amount': 1843},
  {'transaction-id': 1478, 'amount': 2076},
  {'transaction-id': 1561, 'amount': 1644},
  {'transaction-id': 1587, 'amount': 2422},
  {'transaction-id': 2198, 'amount': 2144},
  {'transaction-id': 2415, 'amount': 2082},
  {'transaction-id': 2765, 'amount': 1208},
  {'transaction-id': 2894, 'amount': 1735},
  {'transaction-id': 2993, 'amount': 2189},
  {'transaction-id': 3071, 'amount': 1924},

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 44, 'amount': 2221},
 {'transaction-id': 116, 'amount': 1922},
 {'transaction-id': 211, 'amount': 1793})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(2221, 1922, 1793)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

428.0516218266185

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 108), ('Alice', 108), ('Alice', 117), ('Alice', 117), ('Bob', 83), ('Bob', 84), ('Bob', 90), ('Bob', 91), ('Charlie', 118), ('Charlie', 119), ('Charlie', 127), ('Charlie', 130), ('Dan', 188), ('Dan', 205), ('Dan', 392), ('Edith', 150), ('Edith', 154), ('Edith', 164), ('Edith', 165), ('Frank', 164), ('Frank', 167), ('Frank', 176), ('Frank', 185), ('George', 89), ('George', 90), ('George', 97), ('George', 97), ('Hannah', 131), ('Hannah', 140), ('Hannah', 275), ('Ingrid', 48), ('Ingrid', 48), ('Ingrid', 52), ('Ingrid', 52), ('Jerry', 60), ('Jerry', 60), ('Jerry', 64), ('Jerry', 65), ('Kevin', 152), ('Kevin', 153), ('Kevin', 166), ('Kevin', 167), ('Laura', 117), ('Laura', 117), ('Laura', 126), ('Laura', 128), ('Michael', 67), ('Michael', 70), ('Michael', 75), ('Michael', 75), ('Norbert', 127), ('Norbert', 128), ('Norbert', 236), ('Oliver', 137), ('Oliver', 140), ('Oliver', 151), ('Oliver', 152), ('Patricia', 75), ('Patricia', 76), ('Patricia', 78), ('Patricia', 78), ('Quinn', 12

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 450), ('Bob', 348), ('Charlie', 494), ('Dan', 785), ('Edith', 633), ('Frank', 692), ('George', 373), ('Hannah', 546), ('Ingrid', 200), ('Jerry', 249), ('Kevin', 638), ('Laura', 488), ('Michael', 287), ('Norbert', 491), ('Oliver', 580), ('Patricia', 307), ('Quinn', 500), ('Ray', 349), ('Sarah', 344), ('Tim', 371), ('Ursula', 685), ('Victor', 498), ('Wendy', 480), ('Xavier', 404), ('Yvonne', 500), ('Zelda', 386)]
CPU times: user 128 ms, sys: 8.32 ms, total: 137 ms
Wall time: 472 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Ingrid,"[{'transaction-id': 27, 'amount': 3625}, {'tra..."
1,1,Norbert,"[{'transaction-id': 8801, 'amount': 1413}]"
2,2,Dan,"[{'transaction-id': 251, 'amount': 8}, {'trans..."
3,3,Ingrid,"[{'transaction-id': 721, 'amount': 239}, {'tra..."
4,4,Charlie,"[{'transaction-id': 1803, 'amount': 140}]"


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 159 ms, sys: 16.3 ms, total: 175 ms
Wall time: 1.37 s


name
Alice      450
Bob        348
Charlie    494
Dan        785
Edith      633
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Ingrid', 'amount': 3625, 'transaction-id': 27},
 {'id': 0, 'name': 'Ingrid', 'amount': 3349, 'transaction-id': 184},
 {'id': 0, 'name': 'Ingrid', 'amount': 3467, 'transaction-id': 435})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Ingrid,3625,27
1,0,Ingrid,3349,184
2,0,Ingrid,3467,435
3,0,Ingrid,3332,782
4,0,Ingrid,3134,888


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 132 ms, sys: 21 ms, total: 153 ms
Wall time: 1.05 s


name
Alice       22413
Bob         21742
Charlie     17063
Dan         37134
Edith       29710
Frank       21337
George      11949
Hannah      22090
Ingrid       7145
Jerry       12219
Kevin       13832
Laura       23802
Michael     13409
Norbert     23681
Oliver      26216
Patricia    12661
Quinn       26658
Ray          8998
Sarah        7145
Tim         14826
Ursula      26021
Victor      18055
Wendy       26614
Xavier      22039
Yvonne      26151
Zelda        7090
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()